<a href="https://colab.research.google.com/github/uladzimirkapeika/1c_sales/blob/master/25gb_1c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [1]:
!pip install neptune-client
!pip install neptune-client neptune-contrib[monitoring]

     |████████████████████████████████| 92kB 2.7MB/s 
     |████████████████████████████████| 829kB 9.8MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 204kB 18.9MB/s 
     |████████████████████████████████| 163kB 16.0MB/s 
     |████████████████████████████████| 143kB 18.5MB/s 
     |████████████████████████████████| 71kB 6.2MB/s 
     |████████████████████████████████| 133kB 15.8MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.4.119-py2.py3-none-any.whl size=150019 sha256=ce6fd19b12e3657f6d709bec6722ed3bd28ddea17dcf51167b0231ee169e2f81
  Stored in directory: /root/.cache/pip/wheels/84/5c/c1/a81e80761b94b4467fd3fda1fd3109463702f6247fc422eb33
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=40d63602c5be5778a9147fe1e23ffa860a4fff954172d4e1a6ab754bcc4034f4
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a93

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import product
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.feature_extraction.text import TfidfVectorizer
import neptune
from neptunecontrib.monitoring.lightgbm import neptune_monitor

NEPTUNE_API_TOKEN="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiOGQwMzE5MWYtOWRiNi00NGIyLWIyNWQtMzU3NWFkYmM4ZTVkIn0="

neptune.init('volodyakopeyko/1c-sales', api_token=NEPTUNE_API_TOKEN)
neptune.create_experiment(name='1c_sales')

NVMLError: Driver Not Loaded - GPU usage metrics may not be reported.


https://ui.neptune.ai/volodyakopeyko/1c-sales/e/CSAL-1


Experiment(CSAL-1)

Read data

In [7]:
test = pd.read_csv('/content/drive/My Drive/1C/test.csv')
sales = pd.read_csv('/content/drive/My Drive/1C/sales_train.csv')
shops = pd.read_csv('/content/drive/My Drive/1C//shops.csv')
items = pd.read_csv('/content/drive/My Drive/1C/items.csv')
item_cats = pd.read_csv('/content/drive/My Drive/1C/item_categories.csv')

Shops

In [8]:
shops['city'] = shops['shop_name'].apply(lambda x: x.split()[0].lower())
shops.loc[shops.city == '!якутск', 'city'] = 'якутск'
shops['city_code'] = LabelEncoder().fit_transform(shops['city'])

coords = dict()
coords['якутск'] = (62.028098, 129.732555, 4)
coords['адыгея'] = (44.609764, 40.100516, 3)
coords['балашиха'] = (55.8094500, 37.9580600, 1)
coords['волжский'] = (53.4305800, 50.1190000, 3)
coords['вологда'] = (59.2239000, 39.8839800, 2)
coords['воронеж'] = (51.6720400, 39.1843000, 3)
coords['выездная'] = (0, 0, 0)
coords['жуковский'] = (55.5952800, 38.1202800, 1)
coords['интернет-магазин'] = (0, 0, 0)
coords['казань'] = (55.7887400, 49.1221400, 4)
coords['калуга'] = (54.5293000, 36.2754200, 4)
coords['коломна'] = (55.0794400, 38.7783300, 4)
coords['красноярск'] = (56.0183900, 92.8671700, 4)
coords['курск'] = (51.7373300, 36.1873500, 3)
coords['москва'] = (55.7522200, 37.6155600, 1)
coords['мытищи'] = (55.9116300, 37.7307600, 1)
coords['н.новгород'] = (56.3286700, 44.0020500, 4)
coords['новосибирск'] = (55.0415000, 82.9346000, 4)
coords['омск'] = (54.9924400, 73.3685900, 4)
coords['ростовнадону'] = (47.2313500, 39.7232800, 3)
coords['спб'] = (59.9386300, 30.3141300, 2)
coords['самара'] = (53.2000700, 50.1500000, 4)
coords['сергиев'] = (56.3000000, 38.1333300, 4)
coords['сургут'] = (61.2500000, 73.4166700, 4)
coords['томск'] = (56.4977100, 84.9743700, 4)
coords['тюмень'] = (57.1522200, 65.5272200, 4)
coords['уфа'] = (54.7430600, 55.9677900, 4)
coords['химки'] = (55.8970400, 37.4296900, 1)
coords['цифровой'] = (0, 0, 0)
coords['чехов'] = (55.1477000, 37.4772800, 4)
coords['ярославль'] = (57.6298700, 39.8736800, 2) 

shops['city_coord_1'] = shops['city'].apply(lambda x: coords[x][0])
shops['city_coord_2'] = shops['city'].apply(lambda x: coords[x][1])
shops['country_part'] = shops['city'].apply(lambda x: coords[x][2])

shops = shops[['shop_id', 'city_code', 'city_coord_1', 'city_coord_2', 'country_part']]

In [5]:
shops.head(5)

,shop_id,city_code,city_coord_1,city_coord_2,country_part
0,0,29,62.028098,129.732555,4
1,1,29,62.028098,129.732555,4
2,2,0,44.609764,40.100516,3
3,3,1,55.809450,37.958060,1
4,4,2,53.430580,50.119000,3


In [9]:
items = pd.merge(items, item_cats, on='item_category_id')
items['item_category_name'].unique()
items['item_category'] = items['item_category_name'].apply(lambda x: x.split('-')[0])
items['item_category_common'] = LabelEncoder().fit_transform(items['item_category'])
items['item_category_code'] = LabelEncoder().fit_transform(items['item_category_name'])
items = items[['item_id', 'item_category_common', 'item_category_code']]

# Create dataset

In [10]:
train = sales[sales.item_price < 100000]
train = train[sales.item_cnt_day < 1001]

train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57

train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58

train.loc[train.shop_id == 40, 'shop_id'] = 39
test.loc[test.shop_id == 40, 'shop_id'] = 39

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [11]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

df = [] 
for block_num in train['date_block_num'].unique():
    cur_shops = train.loc[sales['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = train.loc[sales['date_block_num'] == block_num, 'item_id'].unique()
    df.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

df = pd.DataFrame(np.vstack(df), columns = index_cols,dtype=np.int32)
df.head(5)

,shop_id,item_id,date_block_num
0,59,22154,0
1,59,2552,0
2,59,2554,0
3,59,2555,0
4,59,2564,0


In [12]:
#Add month sales
group = train.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day': ['sum']})
group.columns = ['item_cnt_month']
group.reset_index(inplace=True)

df = pd.merge(df, group, on=index_cols, how='left')
df['item_cnt_month'] = (df['item_cnt_month']
                                .fillna(0)
                                .clip(0,20)
                                .astype(np.float16))
df.head(5)

,shop_id,item_id,date_block_num,item_cnt_month
0,59,22154,0,1.0
1,59,2552,0,0.0
2,59,2554,0,0.0
3,59,2555,0,0.0
4,59,2564,0,0.0


In [13]:
test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype(np.int8)
test['shop_id'] = test['shop_id'].astype(np.int8)
test['item_id'] = test['item_id'].astype(np.int16)
df = pd.concat([df, test], ignore_index=True, sort=False, keys=index_cols)
df.fillna(0, inplace=True)
df.head(5)

,shop_id,item_id,date_block_num,item_cnt_month,ID
0,59,22154,0,1.0,0.0
1,59,2552,0,0.0,0.0
2,59,2554,0,0.0,0.0
3,59,2555,0,0.0,0.0
4,59,2564,0,0.0,0.0


In [14]:
# Add items and shops features
df = pd.merge(df, items, on=['item_id'], how='left')
df = pd.merge(df, shops, on=['shop_id'], how='left')

In [15]:
#Calculate intersection features

df1 = df.copy()
first_shop_item_block = df1.groupby(['shop_id', 'item_id'])['date_block_num'].min().reset_index()
first_shop_item_block['shop_item_first_interaction'] = 1

first_item_block = df1.groupby(['item_id'])['date_block_num'].min().reset_index()
first_item_block['item_first_interaction'] = 1

first_shop_item_buy_block = df1[df1['date_block_num'] > 0].groupby(['shop_id', 'item_id'])['date_block_num'].min().reset_index()
first_shop_item_buy_block['first_date_block_num'] = first_shop_item_buy_block['date_block_num']




In [16]:
df = pd.merge(df, first_shop_item_block[['item_id', 'shop_id', 'date_block_num', 'shop_item_first_interaction']], on=['item_id', 'shop_id', 'date_block_num'], how='left')
df = pd.merge(df, first_item_block[['item_id', 'date_block_num', 'item_first_interaction']], on=['item_id', 'date_block_num'], how='left')
df = pd.merge(df, first_shop_item_buy_block[['item_id', 'shop_id', 'first_date_block_num']], on=['item_id', 'shop_id'], how='left')

df['first_date_block_num'].fillna(100, inplace=True)
df['shop_item_sold_before'] = (df['first_date_block_num'] < df['date_block_num']).astype('int8')
df.drop(['first_date_block_num'], axis=1, inplace=True)

df['item_first_interaction'].fillna(0, inplace=True)
df['shop_item_first_interaction'].fillna(0, inplace=True)
df['shop_item_sold_before'].fillna(0, inplace=True)

df['shop_item_first_interaction'] = df['shop_item_first_interaction'].astype('int8')  
df['item_first_interaction'] = df['item_first_interaction'].astype('int8')  
df['shop_item_sold_before'] = df['shop_item_sold_before'].astype('int8') 


In [17]:
def lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
        df[col+'_lag_'+str(i)] = df[col+'_lag_'+str(i)].astype('float32')
    return df

df = lag_feature(df, [1, 2, 3], 'item_cnt_month')
#df.fillna(0, inplace=True)

In [18]:
#avg shop/item price
index_cols = ['shop_id', 'item_id', 'date_block_num']
group = train.groupby(index_cols)['item_price'].mean().reset_index().rename(columns={"item_price": "avg_shop_price"}, errors="raise")
df = pd.merge(df, group, on=index_cols, how='left')

df['avg_shop_price'] = (df['avg_shop_price']
                                .fillna(0)
                                .astype(np.float16))

#avg item price
index_cols = ['item_id', 'date_block_num']
group = train.groupby(['date_block_num','item_id'])['item_price'].mean().reset_index().rename(columns={"item_price": "avg_item_price"}, errors="raise")


df = pd.merge(df, group, on=index_cols, how='left')
df['avg_item_price'] = (df['avg_item_price']
                                .fillna(0)
                                .astype(np.float16))
df['item_shop_price_avg'] = (df['avg_shop_price'] - df['avg_item_price']) / df['avg_item_price']
df = lag_feature(df, [1, 2, 3], 'item_shop_price_avg')
df.head(5)

,shop_id,item_id,date_block_num,item_cnt_month,ID,item_category_common,item_category_code,city_code,city_coord_1,city_coord_2,country_part,shop_item_first_interaction,item_first_interaction,shop_item_sold_before,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,avg_shop_price,avg_item_price,item_shop_price_avg,item_shop_price_avg_lag_1,item_shop_price_avg_lag_2,item_shop_price_avg_lag_3
0,59,22154,0,1.0,0.0,11,37,30,57.62987,39.87368,2,1,1,0,NaN,NaN,NaN,999.0,999.0,0.0,NaN,NaN,NaN
1,59,2552,0,0.0,0.0,13,58,30,57.62987,39.87368,2,1,1,0,NaN,NaN,NaN,0.0,899.0,-1.0,NaN,NaN,NaN
2,59,2554,0,0.0,0.0,13,58,30,57.62987,39.87368,2,1,1,0,NaN,NaN,NaN,0.0,1709.0,-1.0,NaN,NaN,NaN
3,59,2555,0,0.0,0.0,13,56,30,57.62987,39.87368,2,1,1,0,NaN,NaN,NaN,0.0,1099.0,-1.0,NaN,NaN,NaN
4,59,2564,0,0.0,0.0,13,59,30,57.62987,39.87368,2,1,1,0,NaN,NaN,NaN,0.0,327.5,-1.0,NaN,NaN,NaN


In [19]:
group = train.groupby( ["item_id"] ).agg({"item_price": ["mean"]})
group.columns = ["item_avg_item_price"]
group.reset_index(inplace = True)

df = df.merge(group, on = ["item_id"], how = "left" )

group = train.groupby( ["date_block_num","item_id"] ).agg({"item_price": ["mean"]} )
group.columns = ["date_item_avg_item_price"]
group.reset_index(inplace = True)

df = df.merge(group, on = ["date_block_num","item_id"], how = "left")

df['delta_price'] = (df['date_item_avg_item_price'] - df['item_avg_item_price']) / df['item_avg_item_price'] 


In [20]:
df['delta_price'] = (df['delta_price']
                                .fillna(0))

#df = lag_feature(df, [1, 2, 3], 'delta_price')
df.head(5)

,shop_id,item_id,date_block_num,item_cnt_month,ID,item_category_common,item_category_code,city_code,city_coord_1,city_coord_2,country_part,shop_item_first_interaction,item_first_interaction,shop_item_sold_before,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,avg_shop_price,avg_item_price,item_shop_price_avg,item_shop_price_avg_lag_1,item_shop_price_avg_lag_2,item_shop_price_avg_lag_3,item_avg_item_price,date_item_avg_item_price,delta_price
0,59,22154,0,1.0,0.0,11,37,30,57.62987,39.87368,2,1,1,0,NaN,NaN,NaN,999.0,999.0,0.0,NaN,NaN,NaN,702.932203,999.00,0.421190
1,59,2552,0,0.0,0.0,13,58,30,57.62987,39.87368,2,1,1,0,NaN,NaN,NaN,0.0,899.0,-1.0,NaN,NaN,NaN,937.888889,899.00,-0.041464
2,59,2554,0,0.0,0.0,13,58,30,57.62987,39.87368,2,1,1,0,NaN,NaN,NaN,0.0,1709.0,-1.0,NaN,NaN,NaN,1709.050000,1709.05,0.000000
3,59,2555,0,0.0,0.0,13,56,30,57.62987,39.87368,2,1,1,0,NaN,NaN,NaN,0.0,1099.0,-1.0,NaN,NaN,NaN,1123.101786,1098.85,-0.021594
4,59,2564,0,0.0,0.0,13,59,30,57.62987,39.87368,2,1,1,0,NaN,NaN,NaN,0.0,327.5,-1.0,NaN,NaN,NaN,340.152174,327.60,-0.036902


In [21]:
#add date features

import calendar
from datetime import datetime

def count_days(date_block_num):
    year = 2013 + date_block_num // 12
    month = 1 + date_block_num % 12
    weeknd_count = len([1 for i in calendar.monthcalendar(year, month) if i[6] != 0])
    days_in_month = calendar.monthrange(year, month)[1]
    return weeknd_count, days_in_month, month

map_dict = {i: count_days(i) for i in range(35)}

df['weeknd_count'] = df['date_block_num'].apply(lambda x: map_dict[x][0])
df['days_in_month'] = df['date_block_num'].apply(lambda x: map_dict[x][1])
df['month'] = df['date_block_num'].apply(lambda x: map_dict[x][2])

In [19]:
#df = pd.merge(df, sales, on=['date_block_num','shop_id','item_id'], how='left')
#sales['date'] = pd.to_datetime(sales['date'])
#sales['month'] = sales['date'].dt.month

#columns=['month']
#for col in columns:
#    sales[col+'_sin']=np.sin((2*np.pi*sales[col])/max(sales[col]))
#    sales[col+'_cos']=np.cos((2*np.pi*sales[col])/max(sales[col]))

#sales['mean_month_price'] = sales['group_by']
#price_mean_shop = sales.groupby(['date_block_num','shop_id','item_id'])['item_price'].mean().reset_index().rename(columns={"item_price": "price_mean_shop"}, errors="raise")
#price_mean_item = sales.groupby(['date_block_num','item_id'])['item_price'].mean().reset_index().rename(columns={"item_price": "price_mean_item"}, errors="raise")
#sales = pd.merge(sales, price_mean_shop, on=['date_block_num','shop_id','item_id'], how='left')
#sales = pd.merge(sales, price_mean_item, on=['date_block_num','item_id'], how='left')
#sales['month_mean_price'] = 
#price_mean
#sales['mean_month_price'] = sales['mean_month_price']
#grid['item_target_enc'] = grid['item_id'].map(item_id_target_mean)
#grid['item_target_enc'].fillna(0, inplace=True)

In [22]:
item_id_target_mean = df.groupby(['date_block_num','item_id'])['item_cnt_month'].mean().reset_index().rename(columns={"item_cnt_month": "item_target_enc"}, errors="raise")
df = pd.merge(df, item_id_target_mean, on=['date_block_num','item_id'], how='left')

df['item_target_enc'] = (df['item_target_enc']
                                .fillna(0)
                                .astype(np.float16))

df = lag_feature(df, [1, 2, 3], 'item_target_enc')
df.head(5)

,shop_id,item_id,date_block_num,item_cnt_month,ID,item_category_common,item_category_code,city_code,city_coord_1,city_coord_2,country_part,shop_item_first_interaction,item_first_interaction,shop_item_sold_before,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,avg_shop_price,avg_item_price,item_shop_price_avg,item_shop_price_avg_lag_1,item_shop_price_avg_lag_2,item_shop_price_avg_lag_3,item_avg_item_price,date_item_avg_item_price,delta_price,weeknd_count,days_in_month,month,item_target_enc,item_target_enc_lag_1,item_target_enc_lag_2,item_target_enc_lag_3
0,59,22154,0,1.0,0.0,11,37,30,57.62987,39.87368,2,1,1,0,NaN,NaN,NaN,999.0,999.0,0.0,NaN,NaN,NaN,702.932203,999.00,0.421190,4,31,1,0.399902,NaN,NaN,NaN
1,59,2552,0,0.0,0.0,13,58,30,57.62987,39.87368,2,1,1,0,NaN,NaN,NaN,0.0,899.0,-1.0,NaN,NaN,NaN,937.888889,899.00,-0.041464,4,31,1,0.000000,NaN,NaN,NaN
2,59,2554,0,0.0,0.0,13,58,30,57.62987,39.87368,2,1,1,0,NaN,NaN,NaN,0.0,1709.0,-1.0,NaN,NaN,NaN,1709.050000,1709.05,0.000000,4,31,1,0.022217,NaN,NaN,NaN
3,59,2555,0,0.0,0.0,13,56,30,57.62987,39.87368,2,1,1,0,NaN,NaN,NaN,0.0,1099.0,-1.0,NaN,NaN,NaN,1123.101786,1098.85,-0.021594,4,31,1,0.044434,NaN,NaN,NaN
4,59,2564,0,0.0,0.0,13,59,30,57.62987,39.87368,2,1,1,0,NaN,NaN,NaN,0.0,327.5,-1.0,NaN,NaN,NaN,340.152174,327.60,-0.036902,4,31,1,0.111084,NaN,NaN,NaN


In [23]:
item_id_target_mean = df.groupby(['date_block_num','item_id', 'city_code'])['item_cnt_month'].mean().reset_index().rename(columns={
    "item_cnt_month": "item_loc_target_enc"}, errors="raise")
df = pd.merge(df, item_id_target_mean, on=['date_block_num','item_id', 'city_code'], how='left')

df['item_loc_target_enc'] = (df['item_loc_target_enc']
                                .fillna(0)
                                .astype(np.float16))

df = lag_feature(df, [1, 2, 3], 'item_loc_target_enc')
df.head(5)

,shop_id,item_id,date_block_num,item_cnt_month,ID,item_category_common,item_category_code,city_code,city_coord_1,city_coord_2,country_part,shop_item_first_interaction,item_first_interaction,shop_item_sold_before,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,avg_shop_price,avg_item_price,item_shop_price_avg,item_shop_price_avg_lag_1,item_shop_price_avg_lag_2,item_shop_price_avg_lag_3,item_avg_item_price,date_item_avg_item_price,delta_price,weeknd_count,days_in_month,month,item_target_enc,item_target_enc_lag_1,item_target_enc_lag_2,item_target_enc_lag_3,item_loc_target_enc,item_loc_target_enc_lag_1,item_loc_target_enc_lag_2,item_loc_target_enc_lag_3
0,59,22154,0,1.0,0.0,11,37,30,57.62987,39.87368,2,1,1,0,NaN,NaN,NaN,999.0,999.0,0.0,NaN,NaN,NaN,702.932203,999.00,0.421190,4,31,1,0.399902,NaN,NaN,NaN,1.0,NaN,NaN,NaN
1,59,2552,0,0.0,0.0,13,58,30,57.62987,39.87368,2,1,1,0,NaN,NaN,NaN,0.0,899.0,-1.0,NaN,NaN,NaN,937.888889,899.00,-0.041464,4,31,1,0.000000,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2,59,2554,0,0.0,0.0,13,58,30,57.62987,39.87368,2,1,1,0,NaN,NaN,NaN,0.0,1709.0,-1.0,NaN,NaN,NaN,1709.050000,1709.05,0.000000,4,31,1,0.022217,NaN,NaN,NaN,0.0,NaN,NaN,NaN
3,59,2555,0,0.0,0.0,13,56,30,57.62987,39.87368,2,1,1,0,NaN,NaN,NaN,0.0,1099.0,-1.0,NaN,NaN,NaN,1123.101786,1098.85,-0.021594,4,31,1,0.044434,NaN,NaN,NaN,0.0,NaN,NaN,NaN
4,59,2564,0,0.0,0.0,13,59,30,57.62987,39.87368,2,1,1,0,NaN,NaN,NaN,0.0,327.5,-1.0,NaN,NaN,NaN,340.152174,327.60,-0.036902,4,31,1,0.111084,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [24]:
item_id_target_mean = df.groupby(['date_block_num','item_id', 'shop_id'])['item_cnt_month'].mean().reset_index().rename(columns={
    "item_cnt_month": "item_shop_target_enc"}, errors="raise")

df = pd.merge(df, item_id_target_mean, on=['date_block_num','item_id', 'shop_id'], how='left')

df['item_shop_target_enc'] = (df['item_shop_target_enc']
                                .fillna(0)
                                .astype(np.float16))

df = lag_feature(df, [1, 2, 3], 'item_shop_target_enc')
df.head(5)

,shop_id,item_id,date_block_num,item_cnt_month,ID,item_category_common,item_category_code,city_code,city_coord_1,city_coord_2,country_part,shop_item_first_interaction,item_first_interaction,shop_item_sold_before,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,avg_shop_price,avg_item_price,item_shop_price_avg,item_shop_price_avg_lag_1,item_shop_price_avg_lag_2,item_shop_price_avg_lag_3,item_avg_item_price,date_item_avg_item_price,delta_price,weeknd_count,days_in_month,month,item_target_enc,item_target_enc_lag_1,item_target_enc_lag_2,item_target_enc_lag_3,item_loc_target_enc,item_loc_target_enc_lag_1,item_loc_target_enc_lag_2,item_loc_target_enc_lag_3,item_shop_target_enc,item_shop_target_enc_lag_1,item_shop_target_enc_lag_2,item_shop_target_enc_lag_3
0,59,22154,0,1.0,0.0,11,37,30,57.62987,39.87368,2,1,1,0,NaN,NaN,NaN,999.0,999.0,0.0,NaN,NaN,NaN,702.932203,999.00,0.421190,4,31,1,0.399902,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN
1,59,2552,0,0.0,0.0,13,58,30,57.62987,39.87368,2,1,1,0,NaN,NaN,NaN,0.0,899.0,-1.0,NaN,NaN,NaN,937.888889,899.00,-0.041464,4,31,1,0.000000,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2,59,2554,0,0.0,0.0,13,58,30,57.62987,39.87368,2,1,1,0,NaN,NaN,NaN,0.0,1709.0,-1.0,NaN,NaN,NaN,1709.050000,1709.05,0.000000,4,31,1,0.022217,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
3,59,2555,0,0.0,0.0,13,56,30,57.62987,39.87368,2,1,1,0,NaN,NaN,NaN,0.0,1099.0,-1.0,NaN,NaN,NaN,1123.101786,1098.85,-0.021594,4,31,1,0.044434,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
4,59,2564,0,0.0,0.0,13,59,30,57.62987,39.87368,2,1,1,0,NaN,NaN,NaN,0.0,327.5,-1.0,NaN,NaN,NaN,340.152174,327.60,-0.036902,4,31,1,0.111084,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [25]:
#df.fillna(0, inplace=True)
df['delta_count_month_1'] = (df['item_cnt_month_lag_1'] - (df['item_cnt_month_lag_2'])) / (df['item_cnt_month_lag_2'])
df['delta_count_month_2'] = (df['item_cnt_month_lag_2'] - (df['item_cnt_month_lag_3'])) / (df['item_cnt_month_lag_3'])

In [26]:
df[df['delta_count_month_1'] == np.inf]['delta_count_month_1'] = -1
df[df['delta_count_month_2'] == np.inf]['delta_count_month_2'] = -1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [27]:
df.fillna(0, inplace=True)
df = df[(df['date_block_num'] > 2)]
df.head()

,shop_id,item_id,date_block_num,item_cnt_month,ID,item_category_common,item_category_code,city_code,city_coord_1,city_coord_2,country_part,shop_item_first_interaction,item_first_interaction,shop_item_sold_before,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,avg_shop_price,avg_item_price,item_shop_price_avg,item_shop_price_avg_lag_1,item_shop_price_avg_lag_2,item_shop_price_avg_lag_3,item_avg_item_price,date_item_avg_item_price,delta_price,weeknd_count,days_in_month,month,item_target_enc,item_target_enc_lag_1,item_target_enc_lag_2,item_target_enc_lag_3,item_loc_target_enc,item_loc_target_enc_lag_1,item_loc_target_enc_lag_2,item_loc_target_enc_lag_3,item_shop_target_enc,item_shop_target_enc_lag_1,item_shop_target_enc_lag_2,item_shop_target_enc_lag_3,delta_count_month_1,delta_count_month_2
1122795,25,8092,3,3.0,0.0,11,40,13,55.75222,37.61556,1,0,0,1,1.0,0.0,4.0,149.0,146.125,0.019669,0.019669,-1.000000,0.047455,145.754233,146.178571,0.002911,4,30,4,0.608887,0.586914,0.630371,1.444336,0.833496,0.666504,0.666504,2.000000,3.0,1.0,0.0,4.0,inf,-1.000000
1122796,25,7850,3,3.0,0.0,8,30,13,55.75222,37.61556,1,0,0,1,5.0,1.0,6.0,199.0,199.000,0.000000,0.014656,0.025116,0.023132,196.991600,199.000000,0.010195,4,30,4,1.521484,1.869141,1.456055,3.089844,1.916992,2.916016,2.083984,2.916016,3.0,5.0,1.0,6.0,4.0,-0.833333
1122797,25,8051,3,3.0,0.0,14,66,13,55.75222,37.61556,1,0,0,1,0.0,0.0,0.0,30.0,30.000,0.000000,-1.000000,-1.000000,-1.000000,27.083333,30.000000,0.107692,4,30,4,0.108704,0.043488,0.130493,0.133301,0.250000,0.000000,0.333252,0.333252,3.0,0.0,0.0,0.0,0.0,0.000000
1122798,25,8088,3,1.0,0.0,13,55,13,55.75222,37.61556,1,0,0,1,1.0,1.0,0.0,199.0,199.000,0.000000,0.000000,0.283936,-1.000000,186.380952,199.000000,0.067706,4,30,4,0.086975,0.130493,0.043488,0.066650,0.083313,0.166626,0.083313,0.000000,1.0,1.0,1.0,0.0,0.0,inf
1122799,25,8089,3,1.0,0.0,13,55,13,55.75222,37.61556,1,0,0,1,0.0,1.0,0.0,199.0,190.500,0.044617,-1.000000,0.038483,-1.000000,196.164532,190.523810,-0.028755,4,30,4,0.456543,0.478271,0.260986,0.177734,0.333252,0.666504,0.250000,0.083313,1.0,0.0,1.0,0.0,-1.0,inf


Feature engineering

In [28]:
df.columns

Index(['shop_id', 'item_id', 'date_block_num', 'item_cnt_month', 'ID',
       'item_category_common', 'item_category_code', 'city_code',
       'city_coord_1', 'city_coord_2', 'country_part',
       'shop_item_first_interaction', 'item_first_interaction',
       'shop_item_sold_before', 'item_cnt_month_lag_1', 'item_cnt_month_lag_2',
       'item_cnt_month_lag_3', 'avg_shop_price', 'avg_item_price',
       'item_shop_price_avg', 'item_shop_price_avg_lag_1',
       'item_shop_price_avg_lag_2', 'item_shop_price_avg_lag_3',
       'item_avg_item_price', 'date_item_avg_item_price', 'delta_price',
       'weeknd_count', 'days_in_month', 'month', 'item_target_enc',
       'item_target_enc_lag_1', 'item_target_enc_lag_2',
       'item_target_enc_lag_3', 'item_loc_target_enc',
       'item_loc_target_enc_lag_1', 'item_loc_target_enc_lag_2',
       'item_loc_target_enc_lag_3', 'item_shop_target_enc',
       'item_shop_target_enc_lag_1', 'item_shop_target_enc_lag_2',
       'item_shop_target_enc

In [29]:
df.drop(['ID', 
         'avg_shop_price', 
         'avg_item_price', 
         'item_shop_price_avg', 
         'item_avg_item_price', 
         'delta_price_lag', 
         'item_target_enc',
         'item_loc_target_enc',
         'item_shop_target_enc',
         'date_item_avg_item_price',
         'delta_price'
         ], axis=1, inplace=True, errors='ignore')

In [30]:
df['shop_item_sold_before'] = df.apply(lambda x: -1 if x['item_first_interaction'] == 1 else x['shop_item_sold_before'], axis=1)


In [31]:
df.columns

Index(['shop_id', 'item_id', 'date_block_num', 'item_cnt_month',
       'item_category_common', 'item_category_code', 'city_code',
       'city_coord_1', 'city_coord_2', 'country_part',
       'shop_item_first_interaction', 'item_first_interaction',
       'shop_item_sold_before', 'item_cnt_month_lag_1', 'item_cnt_month_lag_2',
       'item_cnt_month_lag_3', 'item_shop_price_avg_lag_1',
       'item_shop_price_avg_lag_2', 'item_shop_price_avg_lag_3',
       'weeknd_count', 'days_in_month', 'month', 'item_target_enc_lag_1',
       'item_target_enc_lag_2', 'item_target_enc_lag_3',
       'item_loc_target_enc_lag_1', 'item_loc_target_enc_lag_2',
       'item_loc_target_enc_lag_3', 'item_shop_target_enc_lag_1',
       'item_shop_target_enc_lag_2', 'item_shop_target_enc_lag_3',
       'delta_count_month_1', 'delta_count_month_2'],
      dtype='object')

In [31]:
X_train = df[df.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = df[df.date_block_num < 33]['item_cnt_month']
X_valid = df[df.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = df[df.date_block_num == 33]['item_cnt_month']
X_test = df[df.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [33]:
feature_name = X_train.columns.tolist()
feature_name.remove('month')
feature_name.remove('shop_item_first_interaction')
#feature_name.remove('shop_item_sold_before')
print(feature_name)

params = {
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 2 ** 7 - 1,
    'learning_rate': 0.005,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.75,
    'bagging_freq': 5,
    'seed': 1,
    'verbose': 1
}

feature_name_indexes = [ 
                        'country_part', 
                        'item_category_common',
                        'item_category_code', 
                        'city_code',
                        #'month'
]

lgb_train = lgb.Dataset(X_train[feature_name], Y_train)
lgb_eval = lgb.Dataset(X_valid[feature_name], Y_valid, reference=lgb_train)

evals_result = {}
gbm = lgb.train(
        params, 
        lgb_train,
        num_boost_round=1000,
        valid_sets=(lgb_train, lgb_eval), 
        feature_name = feature_name,
        categorical_feature = feature_name_indexes,
        verbose_eval=5, 
        evals_result = evals_result,
        early_stopping_rounds = 25,
        callbacks=[neptune_monitor()])


['shop_id', 'item_id', 'date_block_num', 'item_category_common', 'item_category_code', 'city_code', 'city_coord_1', 'city_coord_2', 'country_part', 'item_first_interaction', 'shop_item_sold_before', 'item_cnt_month_lag_1', 'item_cnt_month_lag_2', 'item_cnt_month_lag_3', 'item_shop_price_avg_lag_1', 'item_shop_price_avg_lag_2', 'item_shop_price_avg_lag_3', 'weeknd_count', 'days_in_month', 'item_target_enc_lag_1', 'item_target_enc_lag_2', 'item_target_enc_lag_3', 'item_loc_target_enc_lag_1', 'item_loc_target_enc_lag_2', 'item_loc_target_enc_lag_3', 'item_shop_target_enc_lag_1', 'item_shop_target_enc_lag_2', 'item_shop_target_enc_lag_3', 'delta_count_month_1', 'delta_count_month_2']


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['city_code', 'country_part', 'item_category_code', 'item_category_common']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 25 rounds.
[5]	training's rmse: 1.20402	valid_1's rmse: 1.12746
[10]	training's rmse: 1.19156	valid_1's rmse: 1.11797
[15]	training's rmse: 1.17959	valid_1's rmse: 1.10893
[20]	training's rmse: 1.16786	valid_1's rmse: 1.09995
[25]	training's rmse: 1.15631	valid_1's rmse: 1.09143
[30]	training's rmse: 1.1454	valid_1's rmse: 1.08319
[35]	training's rmse: 1.13499	valid_1's rmse: 1.07542
[40]	training's rmse: 1.12523	valid_1's rmse: 1.06833
[45]	training's rmse: 1.11552	valid_1's rmse: 1.06123
[50]	training's rmse: 1.10624	valid_1's rmse: 1.05439
[55]	training's rmse: 1.09741	valid_1's rmse: 1.04807
[60]	training's rmse: 1.08852	valid_1's rmse: 1.04135
[65]	training's rmse: 1.08025	valid_1's rmse: 1.03521
[70]	training's rmse: 1.07209	valid_1's rmse: 1.02942
[75]	training's rmse: 1.06431	valid_1's rmse: 1.02345
[80]	training's rmse: 1.05682	valid_1's rmse: 1.01791
[85]	training's rmse: 1.0499	valid_1's rmse: 1.01301
[90]	training's rmse: 1

In [54]:
#import joblib
#joblib.dump(gbm, 'lgb_model.pkl')
#neptune.log_artifact('lgb_model.pkl')
neptune.log_metric('valid_best_score', gbm.best_score['valid_1']['rmse'])
neptune.log_text('feature_names', ", ".join(gbm.feature_name()))


In [50]:
pip install neptune-notebooks

     |████████████████████████████████| 1.0MB 3.3MB/s 
  Created wheel for neptune-notebooks: filename=neptune_notebooks-0.0.16-py2.py3-none-any.whl size=1015046 sha256=ab56b74d7ac1f641c689578a80b908901cb90672f696d803c1f8dabc0738e6cb
  Stored in directory: /root/.cache/pip/wheels/b4/3f/5f/7c1ebabf6e91e3eb1aacf265e69420d0a0e2ce879b6af6b8fa
Successfully built neptune-notebooks


In [58]:
!jupyter contrib nbextension install --user

Traceback (most recent call last):
  File "/usr/local/bin/jupyter", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/jupyter_core/command.py", line 230, in main
    command = _jupyter_abspath(subcommand)
  File "/usr/local/lib/python2.7/dist-packages/jupyter_core/command.py", line 133, in _jupyter_abspath
    'Jupyter command `{}` not found.'.format(jupyter_subcommand)
Exception: Jupyter command `jupyter-contrib` not found.


In [59]:
!jupyter nbextension enable neptune-notebooks

Enabling notebook extension neptune-notebooks...
      - Validating: problems found:
        - require?  X neptune-notebooks


In [53]:
!pip install neptune-client

In [49]:
gbm.feature_name()

['shop_id',
 'item_id',
 'date_block_num',
 'item_category_common',
 'item_category_code',
 'city_code',
 'city_coord_1',
 'city_coord_2',
 'country_part',
 'item_first_interaction',
 'shop_item_sold_before',
 'item_cnt_month_lag_1',
 'item_cnt_month_lag_2',
 'item_cnt_month_lag_3',
 'item_shop_price_avg_lag_1',
 'item_shop_price_avg_lag_2',
 'item_shop_price_avg_lag_3',
 'weeknd_count',
 'days_in_month',
 'item_target_enc_lag_1',
 'item_target_enc_lag_2',
 'item_target_enc_lag_3',
 'item_loc_target_enc_lag_1',
 'item_loc_target_enc_lag_2',
 'item_loc_target_enc_lag_3',
 'item_shop_target_enc_lag_1',
 'item_shop_target_enc_lag_2',
 'item_shop_target_enc_lag_3',
 'delta_count_month_1',
 'delta_count_month_2']

In [47]:
dir(gbm)

['_Booster__attr',
 '_Booster__boost',
 '_Booster__get_eval_info',
 '_Booster__higher_better_inner_eval',
 '_Booster__init_predictor',
 '_Booster__inner_eval',
 '_Booster__inner_predict',
 '_Booster__inner_predict_buffer',
 '_Booster__is_predicted_cur_iter',
 '_Booster__name_inner_eval',
 '_Booster__need_reload_eval_info',
 '_Booster__num_class',
 '_Booster__num_dataset',
 '_Booster__num_inner_eval',
 '_Booster__set_objective_to_none',
 '_Booster__train_data_name',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_free_buffer',
 '_to_predictor',
 'add_valid',
 'attr',
 'best_iteration',
 'best_

In [ ]:
№0.896081
-- country part

In [ ]:
#0.897243
#0.895661
#0.900202
#0.900994
#0.900544
#0.901319 with 1
#0.900871 with -1 and 1
#0.902729
#0.899255
0.902628
#0.896363
#!pip install catboost
#from catboost import CatBoostRegressor

#model = CatBoostRegressor(
#    n_estimators = 200,
#    loss_function = 'RMSE',
#    eval_metric = 'RMSE',
#    cat_features = feature_name_indexes )

#model.fit( X_train[feature_name], Y_train.astype('int64'), verbose_eval=5, use_best_model=True, eval_set=(X_valid[feature_name],
#                                                                                                          Y_valid.astype('int64')),
#          plot=True )

In [87]:
#0.897263 
list(zip(gbm.feature_name(),gbm.feature_importance()))

[('shop_id', 2328),
 ('item_id', 5972),
 ('date_block_num', 5471),
 ('item_category_common', 1384),
 ('item_category_code', 5544),
 ('city_code', 3192),
 ('city_coord_1', 316),
 ('city_coord_2', 308),
 ('country_part', 76),
 ('shop_item_first_interaction', 178),
 ('item_first_interaction', 317),
 ('item_cnt_month_lag_1', 1198),
 ('item_cnt_month_lag_2', 1238),
 ('item_cnt_month_lag_3', 988),
 ('item_shop_price_avg_lag_1', 738),
 ('item_shop_price_avg_lag_2', 667),
 ('item_shop_price_avg_lag_3', 259),
 ('delta_price_lag_1', 2498),
 ('delta_price_lag_2', 1074),
 ('delta_price_lag_3', 623),
 ('weeknd_count', 842),
 ('days_in_month', 1153),
 ('item_target_enc_lag_1', 3124),
 ('item_target_enc_lag_2', 1572),
 ('item_target_enc_lag_3', 740),
 ('item_loc_target_enc_lag_1', 528),
 ('item_loc_target_enc_lag_2', 263),
 ('item_loc_target_enc_lag_3', 245),
 ('item_shop_target_enc_lag_1', 340),
 ('item_shop_target_enc_lag_2', 334),
 ('item_shop_target_enc_lag_3', 354),
 ('delta_count_month_1', 459)

In [119]:
#0.898794 kaggle best
#delere fillna before calculating
for col in feature_name:
    print(col, df[col].mean())
list(zip(gbm.feature_name(),gbm.feature_importance()))

shop_id 31.438982121274293
item_id 11297.554650927037
date_block_num 16.93311237690872
item_category_common 11.057187298471975
item_category_code 44.95345177048692
city_code 15.363836568083578
city_coord_1 52.954343977175824
city_coord_2 48.835742765760855
country_part 2.680637967633102
shop_item_first_interaction 0.06597968365976804
item_first_interaction 0.05598993384192975
item_cnt_month_lag_1 0.29191386699676514
item_cnt_month_lag_2 0.29218897223472595
item_cnt_month_lag_3 0.2917971909046173
item_shop_price_avg_lag_1 -0.00019763567252084613
item_shop_price_avg_lag_2 -0.00016628818411845714
item_shop_price_avg_lag_3 -0.00018122559413313866
delta_price_lag_1 -0.01163566019386053
delta_price_lag_2 -0.003641759743914008
delta_price_lag_3 0.00516240019351244
weeknd_count 4.3381911800917345
days_in_month 30.447458605149734
item_target_enc_lag_1 0.2850801646709442
item_target_enc_lag_2 0.2853277325630188
item_target_enc_lag_3 0.28537142276763916
item_loc_target_enc_lag_1 0.290416389703750

[('shop_id', 2406),
 ('item_id', 5925),
 ('date_block_num', 5229),
 ('item_category_common', 1346),
 ('item_category_code', 5460),
 ('city_code', 3164),
 ('city_coord_1', 274),
 ('city_coord_2', 309),
 ('country_part', 69),
 ('shop_item_first_interaction', 167),
 ('item_first_interaction', 298),
 ('item_cnt_month_lag_1', 1298),
 ('item_cnt_month_lag_2', 1169),
 ('item_cnt_month_lag_3', 1106),
 ('item_shop_price_avg_lag_1', 691),
 ('item_shop_price_avg_lag_2', 216),
 ('item_shop_price_avg_lag_3', 99),
 ('delta_price_lag_1', 2368),
 ('delta_price_lag_2', 1020),
 ('delta_price_lag_3', 617),
 ('weeknd_count', 868),
 ('days_in_month', 1184),
 ('item_target_enc_lag_1', 3057),
 ('item_target_enc_lag_2', 1571),
 ('item_target_enc_lag_3', 748),
 ('item_loc_target_enc_lag_1', 568),
 ('item_loc_target_enc_lag_2', 301),
 ('item_loc_target_enc_lag_3', 233),
 ('item_shop_target_enc_lag_1', 365),
 ('item_shop_target_enc_lag_2', 305),
 ('item_shop_target_enc_lag_3', 396),
 ('delta_count_month_1', 791)

In [29]:
#without +1 -1 for delta
#0.901784
for col in feature_name:
    print(col, df[col].mean())
list(zip(gbm.feature_name(),gbm.feature_importance()))

shop_id 31.438982121274293
item_id 11297.554650927037
date_block_num 16.93311237690872
item_category_common 11.057187298471975
item_category_code 44.95345177048692
city_code 15.363836568083578
city_coord_1 52.954343977175824
city_coord_2 48.835742765760855
country_part 2.680637967633102
shop_item_first_interaction 0.06597968365976804
item_first_interaction 0.05598993384192975
item_cnt_month_lag_1 0.29191386699676514
item_cnt_month_lag_2 0.29218897223472595
item_cnt_month_lag_3 0.2917971909046173
item_shop_price_avg_lag_1 -0.00019763567252084613
item_shop_price_avg_lag_2 -0.00016628818411845714
item_shop_price_avg_lag_3 -0.00018122559413313866
delta_price_lag_1 -0.01163566019386053
delta_price_lag_2 -0.003641759743914008
delta_price_lag_3 0.00516240019351244
weeknd_count 4.3381911800917345
days_in_month 30.447458605149734
item_target_enc_lag_1 0.2850801646709442
item_target_enc_lag_2 0.2853277325630188
item_target_enc_lag_3 0.28537142276763916
item_loc_target_enc_lag_1 0.290416389703750

[('shop_id', 1716),
 ('item_id', 4433),
 ('date_block_num', 3797),
 ('item_category_common', 1004),
 ('item_category_code', 4257),
 ('city_code', 2666),
 ('city_coord_1', 242),
 ('city_coord_2', 271),
 ('country_part', 63),
 ('shop_item_first_interaction', 90),
 ('item_first_interaction', 224),
 ('item_cnt_month_lag_1', 1111),
 ('item_cnt_month_lag_2', 973),
 ('item_cnt_month_lag_3', 907),
 ('item_shop_price_avg_lag_1', 396),
 ('item_shop_price_avg_lag_2', 131),
 ('item_shop_price_avg_lag_3', 51),
 ('delta_price_lag_1', 1422),
 ('delta_price_lag_2', 549),
 ('delta_price_lag_3', 285),
 ('weeknd_count', 636),
 ('days_in_month', 822),
 ('item_target_enc_lag_1', 2148),
 ('item_target_enc_lag_2', 1035),
 ('item_target_enc_lag_3', 465),
 ('item_loc_target_enc_lag_1', 343),
 ('item_loc_target_enc_lag_2', 208),
 ('item_loc_target_enc_lag_3', 182),
 ('item_shop_target_enc_lag_1', 364),
 ('item_shop_target_enc_lag_2', 235),
 ('item_shop_target_enc_lag_3', 330),
 ('delta_count_month_1', 716),
 ('

In [ ]:
#without +1 -1 for delta
#0.901784
for col in feature_name:
    print(col, df[col].mean())
list(zip(gbm.feature_name(),gbm.feature_importance()))

In [30]:
#0.896482
for col in feature_name:
    print(col, df[col].mean())
list(zip(gbm.feature_name(),gbm.feature_importance()))

shop_id 31.438982121274293
item_id 11297.554650927037
date_block_num 16.93311237690872
item_category_common 11.057187298471975
item_category_code 44.95345177048692
city_code 15.363836568083578
city_coord_1 52.954343977175824
city_coord_2 48.835742765760855
country_part 2.680637967633102
shop_item_first_interaction 0.06597968365976804
item_first_interaction 0.05598993384192975
item_cnt_month_lag_1 0.29191386699676514
item_cnt_month_lag_2 0.29218897223472595
item_cnt_month_lag_3 0.2917971909046173
item_shop_price_avg_lag_1 -0.6578544974327087
item_shop_price_avg_lag_2 -0.6284127831459045
item_shop_price_avg_lag_3 -0.5996416211128235
delta_price_lag_1 -0.01163566019386053
delta_price_lag_2 -0.003641759743914008
delta_price_lag_3 0.00516240019351244
weeknd_count 4.3381911800917345
days_in_month 30.447458605149734
item_target_enc_lag_1 0.2850801646709442
item_target_enc_lag_2 0.2853277325630188
item_target_enc_lag_3 0.28537142276763916
item_loc_target_enc_lag_1 0.2904163897037506
item_loc_t

[('shop_id', 6136),
 ('item_id', 14283),
 ('date_block_num', 12337),
 ('item_category_common', 3072),
 ('item_category_code', 14427),
 ('city_code', 7130),
 ('city_coord_1', 714),
 ('city_coord_2', 759),
 ('country_part', 156),
 ('shop_item_first_interaction', 541),
 ('item_first_interaction', 768),
 ('item_cnt_month_lag_1', 3529),
 ('item_cnt_month_lag_2', 2512),
 ('item_cnt_month_lag_3', 2237),
 ('item_shop_price_avg_lag_1', 1925),
 ('item_shop_price_avg_lag_2', 1449),
 ('item_shop_price_avg_lag_3', 656),
 ('delta_price_lag_1', 7212),
 ('delta_price_lag_2', 3085),
 ('delta_price_lag_3', 2096),
 ('weeknd_count', 2029),
 ('days_in_month', 2926),
 ('item_target_enc_lag_1', 7901),
 ('item_target_enc_lag_2', 4159),
 ('item_target_enc_lag_3', 2254),
 ('item_loc_target_enc_lag_1', 2005),
 ('item_loc_target_enc_lag_2', 839),
 ('item_loc_target_enc_lag_3', 571),
 ('item_shop_target_enc_lag_1', 927),
 ('item_shop_target_enc_lag_2', 685),
 ('item_shop_target_enc_lag_3', 744),
 ('delta_count_mon

In [36]:
#0.896424
for col in feature_name:
    print(col, df[col].mean())
list(zip(gbm.feature_name(),gbm.feature_importance()))

shop_id 31.438982121274293
item_id 11297.554650927037
date_block_num 16.93311237690872
item_category_common 11.057187298471975
item_category_code 44.95345177048692
city_code 15.363836568083578
city_coord_1 52.954343977175824
city_coord_2 48.835742765760855
country_part 2.680637967633102
shop_item_first_interaction 0.06597968365976804
item_first_interaction 0.05598993384192975
item_cnt_month_lag_1 0.29191386699676514
item_cnt_month_lag_2 0.29218897223472595
item_cnt_month_lag_3 0.2917971909046173
item_shop_price_avg_lag_1 -0.6578544974327087
item_shop_price_avg_lag_2 -0.6284127831459045
item_shop_price_avg_lag_3 -0.5996416211128235
delta_price_lag_1 -0.01163566019386053
delta_price_lag_2 -0.003641759743914008
delta_price_lag_3 0.00516240019351244
weeknd_count 4.3381911800917345
days_in_month 30.447458605149734
item_target_enc_lag_1 0.2850801646709442
item_target_enc_lag_2 0.2853277325630188
item_target_enc_lag_3 0.28537142276763916
item_loc_target_enc_lag_1 0.2904163897037506
item_loc_t

[('shop_id', 5548),
 ('item_id', 13074),
 ('date_block_num', 11453),
 ('item_category_common', 2773),
 ('item_category_code', 13142),
 ('city_code', 6681),
 ('city_coord_1', 666),
 ('city_coord_2', 706),
 ('country_part', 162),
 ('shop_item_first_interaction', 465),
 ('item_first_interaction', 689),
 ('item_cnt_month_lag_1', 3028),
 ('item_cnt_month_lag_2', 2652),
 ('item_cnt_month_lag_3', 2058),
 ('item_shop_price_avg_lag_1', 1785),
 ('item_shop_price_avg_lag_2', 1517),
 ('item_shop_price_avg_lag_3', 538),
 ('delta_price_lag_1', 6222),
 ('delta_price_lag_2', 2675),
 ('delta_price_lag_3', 1730),
 ('weeknd_count', 1898),
 ('days_in_month', 2623),
 ('item_target_enc_lag_1', 7056),
 ('item_target_enc_lag_2', 3795),
 ('item_target_enc_lag_3', 1967),
 ('item_loc_target_enc_lag_1', 1634),
 ('item_loc_target_enc_lag_2', 666),
 ('item_loc_target_enc_lag_3', 476),
 ('item_shop_target_enc_lag_1', 854),
 ('item_shop_target_enc_lag_2', 738),
 ('item_shop_target_enc_lag_3', 646),
 ('delta_count_mon

In [76]:
#0.896363
for col in feature_name:
    print(col, df[col].mean())
list(zip(gbm.feature_name(),gbm.feature_importance()))

shop_id 31.438982121274293
item_id 11297.554650927037
date_block_num 16.93311237690872
item_category_common 11.057187298471975
item_category_code 44.95345177048692
city_code 15.363836568083578
city_coord_1 52.954343977175824
city_coord_2 48.835742765760855
country_part 2.680637967633102
shop_item_first_interaction 0.06597968365976804
item_first_interaction 0.05598993384192975
item_cnt_month_lag_1 0.29191386699676514
item_cnt_month_lag_2 0.29218897223472595
item_cnt_month_lag_3 0.2917971909046173
item_shop_price_avg_lag_1 -0.6578544974327087
item_shop_price_avg_lag_2 -0.6284127831459045
item_shop_price_avg_lag_3 -0.5996416211128235
weeknd_count 4.3381911800917345
days_in_month 30.447458605149734
item_target_enc_lag_1 0.2850801646709442
item_target_enc_lag_2 0.2853277325630188
item_target_enc_lag_3 0.28537142276763916
item_loc_target_enc_lag_1 0.2904163897037506
item_loc_target_enc_lag_2 0.28977128863334656
item_loc_target_enc_lag_3 0.28856533765792847
item_shop_target_enc_lag_1 0.291913

[('shop_id', 6929),
 ('item_id', 16635),
 ('date_block_num', 14155),
 ('item_category_common', 3306),
 ('item_category_code', 15308),
 ('city_code', 7842),
 ('city_coord_1', 866),
 ('city_coord_2', 771),
 ('country_part', 167),
 ('shop_item_first_interaction', 592),
 ('item_first_interaction', 746),
 ('item_cnt_month_lag_1', 3947),
 ('item_cnt_month_lag_2', 2607),
 ('item_cnt_month_lag_3', 2334),
 ('item_shop_price_avg_lag_1', 2787),
 ('item_shop_price_avg_lag_2', 1796),
 ('item_shop_price_avg_lag_3', 1050),
 ('weeknd_count', 2480),
 ('days_in_month', 3262),
 ('item_target_enc_lag_1', 8798),
 ('item_target_enc_lag_2', 4599),
 ('item_target_enc_lag_3', 2854),
 ('item_loc_target_enc_lag_1', 2285),
 ('item_loc_target_enc_lag_2', 1024),
 ('item_loc_target_enc_lag_3', 717),
 ('item_shop_target_enc_lag_1', 1034),
 ('item_shop_target_enc_lag_2', 680),
 ('item_shop_target_enc_lag_3', 846),
 ('delta_count_month_1', 1707),
 ('delta_count_month_2', 268)]

In [43]:
l2 = list(zip(gbm.feature_name(),gbm.feature_importance()))

In [ ]:
214200 
for col in feature_name:
    print(col, sum(X_test[col] == 0) / X_test.shape[0], sum(X_valid[col] == 0) / X_valid.shape[0], sum(X_train[col] == 0) / X_train.shape[0], sum(X_test[col] == 0) / (sum(X_train[col] == 0)+1))

In [37]:
for col in feature_name:
    print(col, sum(df[col] == np.inf))

shop_id 0
item_id 0
date_block_num 0
item_category_common 0
item_category_code 0
city_code 0
city_coord_1 0
city_coord_2 0
country_part 0
item_first_interaction 0
shop_item_sold_before 0
item_cnt_month_lag_1 0
item_cnt_month_lag_2 0
item_cnt_month_lag_3 0
item_shop_price_avg_lag_1 0
item_shop_price_avg_lag_2 0
item_shop_price_avg_lag_3 0
weeknd_count 0
days_in_month 0
item_target_enc_lag_1 0
item_target_enc_lag_2 0
item_target_enc_lag_3 0
item_loc_target_enc_lag_1 0
item_loc_target_enc_lag_2 0
item_loc_target_enc_lag_3 0
item_shop_target_enc_lag_1 0
item_shop_target_enc_lag_2 0
item_shop_target_enc_lag_3 0
delta_count_month_1 642857
delta_count_month_2 634350


In [36]:
X_test[feature_name].fillna(0, inplace=True)
Y_test = gbm.predict(X_test[feature_name]).clip(0, 20)

submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('xgb_submission.csv', index=False)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [ ]:
#with open('best_model_0.87.pkl', 'rb') as f:
#model = pickle.load(open( 'best_model_0.87.pkl', "rb" ))

UnpicklingError: ignored

In [ ]:
Y_pred_test = gbm.predict(X_valid[feature_name]).clip(0, 20)

In [ ]:
gbm.feature_importance()

array([1782, 4327, 3690, 1081, 4159,  104,  218, 1155,  860,  919,  474,
        436,  212, 1458,  561,  269,  620,  840, 2194,  901,  468, 2665,
        246,  278,   77,  323,  219,  199,  269,  210,  250,  617,   49])

In [36]:
gbm.feature_importance()

array([1704, 4445, 3827,  991, 4206, 2606,  259,  273,   64,   96,  224,
        960, 1038,  834,  427,  575,  213, 1460,  599,  285,  638,  807,
       2126, 1025,  413,  325,  194,  183,  323,  275,  304,  313,  118])

In [37]:
gbm.feature_name()

['shop_id',
 'item_id',
 'date_block_num',
 'item_category_common',
 'item_category_code',
 'city_code',
 'city_coord_1',
 'city_coord_2',
 'country_part',
 'shop_item_first_interaction',
 'item_first_interaction',
 'item_cnt_month_lag_1',
 'item_cnt_month_lag_2',
 'item_cnt_month_lag_3',
 'item_shop_price_avg_lag_1',
 'item_shop_price_avg_lag_2',
 'item_shop_price_avg_lag_3',
 'delta_price_lag_lag_1',
 'delta_price_lag_lag_2',
 'delta_price_lag_lag_3',
 'weeknd_count',
 'days_in_month',
 'item_target_enc_lag_1',
 'item_target_enc_lag_2',
 'item_target_enc_lag_3',
 'item_loc_target_enc_lag_1',
 'item_loc_target_enc_lag_2',
 'item_loc_target_enc_lag_3',
 'item_shop_target_enc_lag_1',
 'item_shop_target_enc_lag_2',
 'item_shop_target_enc_lag_3',
 'delta_count_month_1',
 'delta_count_month_2']

In [42]:
gbm1 = ['shop_id',
 'item_id',
 'date_block_num',
 'item_category_common',
 'item_category_code',
 'shop_item_first_interaction',
 'item_first_interaction',
 'item_cnt_month_lag_1',
 'item_cnt_month_lag_2',
 'item_cnt_month_lag_3',
 'item_shop_price_avg_lag_1',
 'item_shop_price_avg_lag_2',
 'item_shop_price_avg_lag_3',
 'delta_price_lag_lag_1',
 'delta_price_lag_lag_2',
 'delta_price_lag_lag_3',
 'weeknd_count',
 'days_in_month',
 'item_target_enc_lag_1',
 'item_target_enc_lag_2',
 'item_target_enc_lag_3',
 'city_code',
 'city_coord_1',
 'city_coord_2',
 'country_part',
 'item_loc_target_enc_lag_1',
 'item_loc_target_enc_lag_2',
 'item_loc_target_enc_lag_3',
 'item_shop_target_enc_lag_1',
 'item_shop_target_enc_lag_2',
 'item_shop_target_enc_lag_3',
 'delta_count_month_1',
 'delta_count_month_2']

l1 = list(zip(gbm1,[1782, 4327, 3690, 1081, 4159,  104,  218, 1155,  860,  919,  474,
        436,  212, 1458,  561,  269,  620,  840, 2194,  901,  468, 2665,
        246,  278,   77,  323,  219,  199,  269,  210,  250,  617,   49]))

In [ ]:
X_train[(X_train['item_first_interaction'] == 1) & (X_train['item_cnt_month_lag_2'] > 0)]

In [ ]:
pd.DataFrame(Y_pred_test)

In [ ]:
df1 = pd.concat([X_valid.reset_index(), Y_valid.reset_index(),pd.DataFrame(Y_pred_test).reset_index()], axis=1)

In [47]:
l3 = [('shop_id', 3349), ('item_id', 7667), ('date_block_num', 6911), ('city_code', 3876), ('city_coord_1', 415), ('city_coord_2', 382), ('country_part', 90), ('item_category_common', 1538), ('item_category_code', 8124), ('shop_item_first_interaction', 345), ('item_first_interaction', 426), ('item_cnt_month_lag_1', 2154), ('item_cnt_month_lag_2', 1379), ('item_cnt_month_lag_3', 1268), ('item_shop_price_avg_lag_1', 1181), ('item_shop_price_avg_lag_2', 812), ('item_shop_price_avg_lag_3', 340), ('delta_price_lag_lag_1', 4173), ('delta_price_lag_lag_2', 1854), ('delta_price_lag_lag_3', 1269), ('weeknd_count', 1097), ('days_in_month', 1648), ('item_target_enc_lag_1', 4576), ('item_target_enc_lag_2', 2373), ('item_target_enc_lag_3', 1368), ('item_loc_target_enc_lag_1', 1173), ('item_loc_target_enc_lag_2', 459), ('item_loc_target_enc_lag_3', 306), ('item_shop_target_enc_lag_1', 537), ('item_shop_target_enc_lag_2', 338), ('item_shop_target_enc_lag_3', 348), ('delta_count_month_1', 776), ('delta_count_month_2', 70)]

In [48]:
l3

[('shop_id', 3349),
 ('item_id', 7667),
 ('date_block_num', 6911),
 ('city_code', 3876),
 ('city_coord_1', 415),
 ('city_coord_2', 382),
 ('country_part', 90),
 ('item_category_common', 1538),
 ('item_category_code', 8124),
 ('shop_item_first_interaction', 345),
 ('item_first_interaction', 426),
 ('item_cnt_month_lag_1', 2154),
 ('item_cnt_month_lag_2', 1379),
 ('item_cnt_month_lag_3', 1268),
 ('item_shop_price_avg_lag_1', 1181),
 ('item_shop_price_avg_lag_2', 812),
 ('item_shop_price_avg_lag_3', 340),
 ('delta_price_lag_lag_1', 4173),
 ('delta_price_lag_lag_2', 1854),
 ('delta_price_lag_lag_3', 1269),
 ('weeknd_count', 1097),
 ('days_in_month', 1648),
 ('item_target_enc_lag_1', 4576),
 ('item_target_enc_lag_2', 2373),
 ('item_target_enc_lag_3', 1368),
 ('item_loc_target_enc_lag_1', 1173),
 ('item_loc_target_enc_lag_2', 459),
 ('item_loc_target_enc_lag_3', 306),
 ('item_shop_target_enc_lag_1', 537),
 ('item_shop_target_enc_lag_2', 338),
 ('item_shop_target_enc_lag_3', 348),
 ('delta_co

In [44]:
l1

[('shop_id', 1782),
 ('item_id', 4327),
 ('date_block_num', 3690),
 ('item_category_common', 1081),
 ('item_category_code', 4159),
 ('shop_item_first_interaction', 104),
 ('item_first_interaction', 218),
 ('item_cnt_month_lag_1', 1155),
 ('item_cnt_month_lag_2', 860),
 ('item_cnt_month_lag_3', 919),
 ('item_shop_price_avg_lag_1', 474),
 ('item_shop_price_avg_lag_2', 436),
 ('item_shop_price_avg_lag_3', 212),
 ('delta_price_lag_lag_1', 1458),
 ('delta_price_lag_lag_2', 561),
 ('delta_price_lag_lag_3', 269),
 ('weeknd_count', 620),
 ('days_in_month', 840),
 ('item_target_enc_lag_1', 2194),
 ('item_target_enc_lag_2', 901),
 ('item_target_enc_lag_3', 468),
 ('city_code', 2665),
 ('city_coord_1', 246),
 ('city_coord_2', 278),
 ('country_part', 77),
 ('item_loc_target_enc_lag_1', 323),
 ('item_loc_target_enc_lag_2', 219),
 ('item_loc_target_enc_lag_3', 199),
 ('item_shop_target_enc_lag_1', 269),
 ('item_shop_target_enc_lag_2', 210),
 ('item_shop_target_enc_lag_3', 250),
 ('delta_count_month_

In [45]:
l2

[('shop_id', 1704),
 ('item_id', 4445),
 ('date_block_num', 3827),
 ('item_category_common', 991),
 ('item_category_code', 4206),
 ('city_code', 2606),
 ('city_coord_1', 259),
 ('city_coord_2', 273),
 ('country_part', 64),
 ('shop_item_first_interaction', 96),
 ('item_first_interaction', 224),
 ('item_cnt_month_lag_1', 960),
 ('item_cnt_month_lag_2', 1038),
 ('item_cnt_month_lag_3', 834),
 ('item_shop_price_avg_lag_1', 427),
 ('item_shop_price_avg_lag_2', 575),
 ('item_shop_price_avg_lag_3', 213),
 ('delta_price_lag_lag_1', 1460),
 ('delta_price_lag_lag_2', 599),
 ('delta_price_lag_lag_3', 285),
 ('weeknd_count', 638),
 ('days_in_month', 807),
 ('item_target_enc_lag_1', 2126),
 ('item_target_enc_lag_2', 1025),
 ('item_target_enc_lag_3', 413),
 ('item_loc_target_enc_lag_1', 325),
 ('item_loc_target_enc_lag_2', 194),
 ('item_loc_target_enc_lag_3', 183),
 ('item_shop_target_enc_lag_1', 323),
 ('item_shop_target_enc_lag_2', 275),
 ('item_shop_target_enc_lag_3', 304),
 ('delta_count_month_1

In [ ]:
df[df['item_id'] == 7223]

In [ ]:
df1[df1['item_id'] == 10201][feature_name].head()

,shop_id,item_id,date_block_num,item_category_common,item_category_code,shop_item_first_interaction,item_first_interaction,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_shop_price_avg_lag_1,item_shop_price_avg_lag_2,item_shop_price_avg_lag_3,delta_price_lag_lag_1,delta_price_lag_lag_2,delta_price_lag_lag_3,weeknd_count,days_in_month,item_target_enc_lag_1,item_target_enc_lag_2,item_target_enc_lag_3,city_code,city_coord_1,city_coord_2,country_part,item_loc_target_enc_lag_1,item_loc_target_enc_lag_2,item_loc_target_enc_lag_3,item_shop_target_enc_lag_1,item_shop_target_enc_lag_2,item_shop_target_enc_lag_3,delta_count_month_1,delta_count_month_2
448,45,10201,33,8,28,1,1,2.646484,2.154297,2.076172,0.0,0.0,0.0,0.0,0.0,0.0,4,31,2.646484,2.154297,2.076172,19,53.20007,50.15000,4,2.646484,2.154297,2.076172,2.646484,2.154297,2.076172,0.228468,0.037629
5861,46,10201,33,8,28,1,1,2.646484,2.154297,2.076172,0.0,0.0,0.0,0.0,0.0,0.0,4,31,2.646484,2.154297,2.076172,20,56.30000,38.13333,4,2.646484,2.154297,2.076172,2.646484,2.154297,2.076172,0.228468,0.037629
11274,44,10201,33,8,28,1,1,2.646484,2.154297,2.076172,0.0,0.0,0.0,0.0,0.0,0.0,4,31,2.646484,2.154297,2.076172,19,53.20007,50.15000,4,2.646484,2.154297,2.076172,2.646484,2.154297,2.076172,0.228468,0.037629
16687,41,10201,33,8,28,1,1,2.646484,2.154297,2.076172,0.0,0.0,0.0,0.0,0.0,0.0,4,31,2.646484,2.154297,2.076172,18,47.23135,39.72328,3,2.646484,2.154297,2.076172,2.646484,2.154297,2.076172,0.228468,0.037629
22100,39,10201,33,8,28,1,1,2.646484,2.154297,2.076172,0.0,0.0,0.0,0.0,0.0,0.0,4,31,2.646484,2.154297,2.076172,18,47.23135,39.72328,3,2.646484,2.154297,2.076172,2.646484,2.154297,2.076172,0.228468,0.037629


In [ ]:
df1.groupby('item_id')['rmse'].sum().sort_values()

item_id
4184        0.016337
564         0.017951
871         0.017951
889         0.018122
16285       0.018828
            ...     
7224     2636.565573
1583     2892.055238
10202    4895.895121
7223     6268.910623
10201    7888.834109
Name: rmse, Length: 5413, dtype: float64

In [ ]:
df1.groupby('item_id')['rmse'].sum().sort_values()

In [ ]:
items[items['item_id'] == 1583] 

In [ ]:
#from xgboost import XGBRegressor
#model = XGBRegressor(
#    max_depth=10,
#    min_child_weight=0.5, 
#    colsample_bytree=0.8, 
#    subsample=0.8, 
#    eta=0.1,
#     tree_method='gpu_hist',
#    seed=42)

#model.fit(
#    X_train, 
#    Y_train, 
#    eval_metric="rmse", 
#    eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
#    verbose=True, 
#    early_stopping_rounds = 20)


In [ ]:
feature_name

In [ ]:
#0.935226
#0.935715
#0.931289
#0.918754
#0.924846
#0.924846
#0.923147
#0.917676
#0.914308 
#0.912568
#0.916846
#0.916665
#0.912285
#0.914761 cat mean enc
#0.915836
#0.910541
#0.914833
#0.904
#0.909211
#0.909396

In [ ]:
lgb.plot_importance(
    gbm, 
    max_num_features=50, 
    importance_type='split', 
    figsize=(12,8));

In [ ]:
lgb.plot_importance(
    gbm, 
    max_num_features=50, 
    importance_type='gain', 
    figsize=(12,8));

In [ ]:
lgb.plot_importance(
    gbm, 
    max_num_features=50, 
    importance_type='gain', 
    figsize=(12,8));

In [ ]:
#items_subset = items[['item_id', 'item_name']]
#feature_count = 35
#tfidf = TfidfVectorizer(max_features=feature_count)
#items_df_item_name_text_features = pd.DataFrame(tfidf.fit_transform(items_subset['item_name']).toarray())

#cols = items_df_item_name_text_features.columns
#for i in range(feature_count):
#    feature_name = 'item_name_tfidf_' + str(i)
#    items_subset[feature_name] = items_df_item_name_text_features[cols[i]]

#items_subset.drop('item_name', axis = 1, inplace = True)